# FDR

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from functions import *

In [ ]:
import ray
ray.init()

In [ ]:
np.logspace( -7, -2, 6 )

In [ ]:
mu     = .036 
sigma  = .079
T      = 24
gamma3 = -2.448
gamma4 = 10.164
SR0    = 0
SR1    = .5
p_H1   = .05
alpha  = .05
q      = .25

number_of_trials = 10
variance = .1
SR0_adj = expected_maximum_sharpe_ratio( number_of_trials, variance )

xs = np.unique( sorted( np.hstack( [
    # More points close to zero, to show the vertical asymptote
    np.logspace( -12, -2, 11 ),
    np.linspace( 0, .01, 11 ),
    np.linspace( 0, 1, 101 ),
] ) ) )[1:]

@ray.remote
def control_for_FDR_ray( q, SR0, SR1, p_H1, T, gamma3, gamma4 ):
    return control_for_FDR( q = q, SR0 = SR0, SR1 = SR1, p_H1 = p_H1, T = T, gamma3 = gamma3, gamma4 = gamma4 )

ys = [ 
    control_for_FDR_ray.remote( q = q, SR0 = SR0, SR1 = SR1, p_H1 = p_H1, T = T, gamma3 = gamma3, gamma4 = gamma4 )
    for p_H1 in xs
]
ys = [ ray.get( y ) for y in tqdm(ys) ]
ys = pd.DataFrame( np.array( ys ), index = xs, columns = ['alpha', 'beta', 'SR_c'] )

ys2 = [ 
    control_for_FDR_ray.remote( q = q, SR0 = SR0_adj, SR1 = SR1 + SR0_adj, p_H1 = p_H1, T = T, gamma3 = gamma3, gamma4 = gamma4 )
    for p_H1 in xs
]
ys2 = [ ray.get( y ) for y in tqdm(ys2) ]
ys2 = pd.DataFrame( np.array( ys2 ), index = xs, columns = ['alpha', 'beta', 'SR_c'] )


In [ ]:
fig, axs = plt.subplots( 1, 3, figsize = (9,2.5), layout = 'constrained' )
axs[0].plot( ys.index, ys['alpha'], label = 'alpha' )
axs[1].plot( ys.index, ys['beta'], label = 'beta' )
axs[2].plot( ys.index, ys['SR_c'], label = 'SR_c' )
axs[2].plot( ys2.index, ys2['SR_c'], label = 'SR_c_adj' )
axs[0].set_ylabel( "alpha" )
axs[1].set_ylabel( "beta" )
axs[2].set_ylabel( "SR_c" )
axs[2].set_ylim( -.5, 1.6 )
for ax in axs: 
    ax.set_xlabel( "p[H1]" )
    ax.axhline( 0, color = 'black', linestyle = ':', linewidth = 1 )
    ax.set_xlim(0, 1)
    ax.axvline( 1-q, color = 'black', linestyle = '--', linewidth = 1 )
fig.suptitle( 
    f"SR0 = {SR0}, SR1 = {SR1}, q = {q}"
    "\n"
    f"SR0_adj = {SR0_adj:.3f}, SR1_adj = {SR1 + SR0_adj:.3f}"
)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize = (5,3.5), layout = 'constrained', dpi = 300)
ax.plot( ys.index, ys['SR_c'],   linewidth = 3, label = f"SR0 = {SR0}, SR1 = {SR1}" )
ax.plot( ys2.index, ys2['SR_c'], linewidth = 3, label = f"SR0 = {SR0_adj:.3f}, SR1 = {SR1 + SR0_adj:.3f}" )
ax.set_ylabel( "SR_c" )
ax.set_xlabel( "p[H1]" )
ax.axhline( 0, color = 'black', linestyle = ':', linewidth = 1 )
ax.set_xlim(0, 1)
ax.set_title( f"SR_c vs P[H1] (q={q})" )
ax.legend()
ax.axvline( 1-q, color = 'black', linestyle = '--', linewidth = 1 )
ax.set_ylim( -.5, 1.6 )
plt.show()